In [43]:
import sqlite3
import csv
import pandas as pd
import os
import re

import math
from tqdm import tqdm
import multiprocessing as mp
import time

import matplotlib.pyplot as plt

import numpy as np

folder_prefix = '/home/jovyan/work/'



In [44]:
new_data = pd.read_pickle(folder_prefix+ 'charters_counts_2015.pkl')

In [48]:
new_data

,CMO_NUMSTATES,CMO_SIZE,LEVEL,LOCALE15,PVI2017,PVI2014,RLAPROF_1516,MTHPROF_1516,ess_strength,prog_strength,...,RIT_COUNT,ESS_COUNT,ESS_STR,PROG_STR,RIT_STR,NUMWORDS,PROG_NORM,ESS_NORM,RIT_NORM,PCT_CHILDPOV_SAIPE16
0,1.0,NaN,3,Suburb (midsize),-23.0,-16.0,59.0,34.0,0.087500,0.037500,...,7,0,-6.000000,-2.977266,-2.132168,949,0.624857,0.000000,0.791405,NaN
1,1.0,1220.0,3,Suburb (midsize),-23.0,-16.0,59.0,34.0,0.087500,0.037500,...,7,0,-6.000000,-2.977266,-2.132168,949,0.624857,0.000000,0.791405,NaN
2,1.0,NaN,3,City (large),-23.0,-16.0,44.0,34.0,0.092505,0.097047,...,8,0,-6.000000,-2.941014,-2.037426,873,0.632351,0.000000,0.810791,NaN
3,1.0,1220.0,3,City (large),-23.0,-16.0,44.0,34.0,0.092505,0.097047,...,8,0,-6.000000,-2.941014,-2.037426,873,0.632351,0.000000,0.810791,NaN
4,1.0,NaN,3,City (large),9.0,12.0,59.0,49.0,0.083333,0.020833,...,1,0,-6.000000,-6.000000,-2.664642,462,0.000000,0.000000,0.682455,NaN
5,1.0,1220.0,3,City (large),9.0,12.0,59.0,49.0,0.083333,0.020833,...,1,0,-6.000000,-6.000000,-2.664642,462,0.000000,0.000000,0.682455,NaN
6,1.0,NaN,2,City (midsize),25.0,24.0,78.0,77.0,0.062500,0.234375,...,706,192,-2.547275,-2.141070,-1.981662,67698,0.797715,0.758551,0.822201,NaN
7,1.0,NaN,4,Rural (fringe),NaN,NaN,74.0,89.0,NaN,NaN,...,140,28,-3.420362,-3.299189,-2.721245,73710,0.558309,0.566737,0.670873,NaN
8,1.0,NaN,1,City (midsize),25.0,24.0,77.0,73.0,0.062500,0.234375,...,826,13,-3.669989,-2.017585,-1.866895,60804,0.823241,0.511895,0.845683,NaN
9,1.0,NaN,1,City (large),-29.0,-25.0,79.0,89.0,0.224490,0.081633,...,55,6,-2.871767,-2.649335,-1.909070,4466,0.692647,0.687261,0.837054,NaN


In [4]:
new_data["high_ess"] = 0
new_data["high_prog"] = 0
new_data["high_both"] = 0
new_data["low_both"] = 0

new_data['high_ess'] = new_data['high_ess'].astype('object')
new_data['high_prog'] = new_data['high_prog'].astype('object')
new_data['high_both'] = new_data['high_both'].astype('object')
new_data['low_both'] = new_data['low_both'].astype('object')

In [5]:
new_data

,CMO_NUMSTATES,CMO_SIZE,LEVEL,LOCALE15,PVI2017,PVI2014,RLAPROF_1516,MTHPROF_1516,ess_strength,prog_strength,...,RIT_STR,NUMWORDS,PROG_NORM,ESS_NORM,RIT_NORM,PCT_CHILDPOV_SAIPE16,high_ess,high_prog,high_both,low_both
0,1.0,NaN,3,Suburb (midsize),-23.0,-16.0,59.0,34.0,0.087500,0.037500,...,-2.132168,949,0.624857,0.000000,0.791405,NaN,0,0,0,0
1,1.0,1220.0,3,Suburb (midsize),-23.0,-16.0,59.0,34.0,0.087500,0.037500,...,-2.132168,949,0.624857,0.000000,0.791405,NaN,0,0,0,0
2,1.0,NaN,3,City (large),-23.0,-16.0,44.0,34.0,0.092505,0.097047,...,-2.037426,873,0.632351,0.000000,0.810791,NaN,0,0,0,0
3,1.0,1220.0,3,City (large),-23.0,-16.0,44.0,34.0,0.092505,0.097047,...,-2.037426,873,0.632351,0.000000,0.810791,NaN,0,0,0,0
4,1.0,NaN,3,City (large),9.0,12.0,59.0,49.0,0.083333,0.020833,...,-2.664642,462,0.000000,0.000000,0.682455,NaN,0,0,0,0
5,1.0,1220.0,3,City (large),9.0,12.0,59.0,49.0,0.083333,0.020833,...,-2.664642,462,0.000000,0.000000,0.682455,NaN,0,0,0,0
6,1.0,NaN,2,City (midsize),25.0,24.0,78.0,77.0,0.062500,0.234375,...,-1.981662,67698,0.797715,0.758551,0.822201,NaN,0,0,0,0
7,1.0,NaN,4,Rural (fringe),NaN,NaN,74.0,89.0,NaN,NaN,...,-2.721245,73710,0.558309,0.566737,0.670873,NaN,0,0,0,0
8,1.0,NaN,1,City (midsize),25.0,24.0,77.0,73.0,0.062500,0.234375,...,-1.866895,60804,0.823241,0.511895,0.845683,NaN,0,0,0,0
9,1.0,NaN,1,City (large),-29.0,-25.0,79.0,89.0,0.224490,0.081633,...,-1.909070,4466,0.692647,0.687261,0.837054,NaN,0,0,0,0


In [6]:
num_rows = new_data.shape[0] 
num_rows

7294

In [7]:
new_data['PROG_STR']

0       -2.977266
1       -2.977266
2       -2.941014
3       -2.941014
4       -6.000000
5       -6.000000
6       -2.141070
7       -3.299189
8       -2.017585
9       -2.649335
10      -2.649335
11      -2.649335
12      -2.649335
13      -2.149086
14      -2.149086
15      -2.149086
16      -2.149086
17      -2.619293
18      -2.556303
19      -6.000000
20      -2.963492
21      -6.000000
22      -6.000000
23      -6.000000
24      -2.968074
25      -2.968074
26      -2.267954
28      -6.000000
29      -6.000000
30      -6.000000
           ...   
11368   -2.338519
11369   -6.000000
11370   -6.000000
11371   -3.079078
11372   -2.367356
11373   -3.303466
11375   -2.514688
11376   -3.230181
11377   -3.398764
11378   -6.000000
11379   -6.000000
11380   -2.757486
11381   -2.756470
11382   -2.811724
11383   -2.162863
11384   -3.199206
11386   -6.000000
11387   -2.633650
11388   -2.653213
11389   -2.567716
11390   -2.519450
11391   -2.633433
11392   -2.387187
11393   -2.202259
11394   -2

In [8]:
new_data['ESS_STR']

0       -6.000000
1       -6.000000
2       -6.000000
3       -6.000000
4       -6.000000
5       -6.000000
6       -2.547275
7       -3.420362
8       -3.669989
9       -2.871767
10      -2.871767
11      -2.871767
12      -2.871767
13      -2.913125
14      -2.913125
15      -2.913125
16      -2.913125
17      -3.638830
18      -6.000000
19      -6.000000
20      -3.259497
21      -6.000000
22      -2.226600
23      -2.226600
24      -2.278636
25      -2.278636
26      -2.444045
28      -2.188410
29      -6.000000
30      -6.000000
           ...   
11368   -2.673900
11369   -6.000000
11370   -6.000000
11371   -2.845046
11372   -6.000000
11373   -2.864295
11375   -2.831998
11376   -2.996232
11377   -2.523660
11378   -6.000000
11379   -6.000000
11380   -3.103925
11381   -3.104611
11382   -3.067443
11383   -6.000000
11384   -2.721536
11386   -6.000000
11387   -2.793868
11388   -2.654177
11389   -2.769499
11390   -2.990183
11391   -2.421321
11392   -2.910408
11393   -3.076666
11394   -2

In [9]:
prog_10 = np.percentile(new_data['PROG_STR'].fillna(0), 10) #10th percentile means in 10% of scores are below yours and 90% above yours
prog_90 = np.percentile(new_data['PROG_STR'].fillna(0), 90)
print(prog_10)
print(prog_90)
#max(new_data['PROG_STR'])

-6.0
-2.27088390037


In [10]:
ess_10 = np.percentile(new_data['ESS_STR'].fillna(0), 10) #10th percentile means in 10% of scores are below yours and 90% above yours
ess_90 = np.percentile(new_data['ESS_STR'].fillna(0), 90)
print(ess_10)
print(ess_90)

-6.0
-2.34763296393


In [11]:
new_data['ESS_STR'][0] > ess_90

False

In [12]:
for index, row in new_data.iterrows():
    
    if(row['ESS_STR'] >= ess_90) and (row['PROG_STR'] <= prog_10):
        new_data.loc[index, 'high_ess'] = 1
    if(row['PROG_STR'] >= prog_90) and (row['ESS_STR'] <= ess_10):
        new_data.loc[index, 'high_prog'] = 1
    if(row['ESS_STR'] >= ess_90) and (row['PROG_STR'] >= prog_90):
        new_data.loc[index, 'high_both'] = 1
    if(row['ESS_STR'] <= ess_10) and (row['PROG_STR'] <= prog_10):
        new_data.loc[index, 'low_both'] = 1


In [13]:
new_data

,CMO_NUMSTATES,CMO_SIZE,LEVEL,LOCALE15,PVI2017,PVI2014,RLAPROF_1516,MTHPROF_1516,ess_strength,prog_strength,...,RIT_STR,NUMWORDS,PROG_NORM,ESS_NORM,RIT_NORM,PCT_CHILDPOV_SAIPE16,high_ess,high_prog,high_both,low_both
0,1.0,NaN,3,Suburb (midsize),-23.0,-16.0,59.0,34.0,0.087500,0.037500,...,-2.132168,949,0.624857,0.000000,0.791405,NaN,0,0,0,0
1,1.0,1220.0,3,Suburb (midsize),-23.0,-16.0,59.0,34.0,0.087500,0.037500,...,-2.132168,949,0.624857,0.000000,0.791405,NaN,0,0,0,0
2,1.0,NaN,3,City (large),-23.0,-16.0,44.0,34.0,0.092505,0.097047,...,-2.037426,873,0.632351,0.000000,0.810791,NaN,0,0,0,0
3,1.0,1220.0,3,City (large),-23.0,-16.0,44.0,34.0,0.092505,0.097047,...,-2.037426,873,0.632351,0.000000,0.810791,NaN,0,0,0,0
4,1.0,NaN,3,City (large),9.0,12.0,59.0,49.0,0.083333,0.020833,...,-2.664642,462,0.000000,0.000000,0.682455,NaN,0,0,0,1
5,1.0,1220.0,3,City (large),9.0,12.0,59.0,49.0,0.083333,0.020833,...,-2.664642,462,0.000000,0.000000,0.682455,NaN,0,0,0,1
6,1.0,NaN,2,City (midsize),25.0,24.0,78.0,77.0,0.062500,0.234375,...,-1.981662,67698,0.797715,0.758551,0.822201,NaN,0,0,0,0
7,1.0,NaN,4,Rural (fringe),NaN,NaN,74.0,89.0,NaN,NaN,...,-2.721245,73710,0.558309,0.566737,0.670873,NaN,0,0,0,0
8,1.0,NaN,1,City (midsize),25.0,24.0,77.0,73.0,0.062500,0.234375,...,-1.866895,60804,0.823241,0.511895,0.845683,NaN,0,0,0,0
9,1.0,NaN,1,City (large),-29.0,-25.0,79.0,89.0,0.224490,0.081633,...,-1.909070,4466,0.692647,0.687261,0.837054,NaN,0,0,0,0


In [14]:
#new_data.loc[11399, 'high_ess'] = 1

In [15]:
np.count_nonzero(new_data['high_ess'])

75

In [16]:
np.count_nonzero(new_data['high_prog'])

99

In [17]:
np.count_nonzero(new_data['high_both'])

112

In [18]:
np.count_nonzero(new_data['low_both'])

1070

In [19]:
trouble_indices = []
for index, row in new_data.iterrows():
    if(row['high_ess'] == 0) and (row['high_prog'] == 0) and (row['high_both'] == 0) and (row['low_both'] == 0):
        trouble_indices.append(index)
        print(index)
    


0
1
2
3
6
7
8
9
10
11
12
13
14
15
16
17
18
20
24
25
26
41
42
43
61
62
69
70
161
162
184
185
198
199
200
201
210
211
216
221
222
223
224
225
227
228
229
230
233
234
235
236
237
238
239
240
241
242
244
245
246
247
248
249
250
251
254
255
256
257
258
259
260
261
262
263
264
265
266
267
269
270
271
273
274
275
277
278
279
280
281
282
283
284
285
286
289
290
292
293
294
295
296
297
298
299
300
301
302
304
305
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
353
354
355
356
357
358
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
387
389
390
391
392
393
394
395
396
399
401
402
403
405
407
409
411
412
413
414
416
417
418
420
421
423
424
426
428
432
433
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
468
470
471
472
473
475
479
480
483
484
485
486
487
490


2580
2581
2582
2583
2584
2585
2586
2587
2588
2589
2590
2591
2592
2593
2594
2595
2596
2608
2609
2610
2611
2612
2613
2614
2615
2616
2617
2621
2622
2623
2624
2626
2627
2628
2629
2632
2633
2637
2638
2639
2640
2641
2643
2644
2646
2647
2648
2650
2651
2652
2653
2654
2655
2656
2657
2658
2659
2660
2661
2662
2669
2681
2682
2683
2684
2685
2686
2688
2689
2690
2691
2692
2696
2697
2698
2699
2702
2703
2704
2705
2707
2709
2711
2712
2713
2714
2715
2716
2717
2718
2719
2720
2721
2722
2723
2724
2725
2726
2727
2728
2729
2730
2731
2732
2733
2734
2735
2736
2737
2738
2739
2740
2741
2742
2743
2744
2745
2746
2747
2748
2749
2750
2751
2753
2754
2755
2756
2758
2759
2760
2761
2762
2763
2764
2765
2769
2770
2778
2781
2784
2785
2786
2787
2788
2789
2790
2794
2795
2799
2800
2801
2802
2803
2804
2805
2806
2807
2808
2809
2810
2811
2812
2813
2814
2815
2816
2817
2818
2819
2820
2821
2822
2823
2824
2825
2826
2827
2828
2829
2830
2831
2836
2837
2838
2843
2844
2845
2846
2847
2848
2849
2850
2851
2852
2853
2854
2856
2857
2858
2859


8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8577
8579
8582
8585
8586
8588
8589
8590
8591
8592
8594
8595
8597
8598
8599
8600
8601
8604
8605
8606
8608
8609
8610
8611
8613
8614
8615
8616
8618
8619
8620
8621
8623
8624
8626
8627
8630
8631
8632
8633
8634
8635
8636
8637
8638
8639
8641
8642
8644
8645
8646
8647
8648
8649
8650
8651
8652
8653
8654
8655
8656
8658
8659
8660
8661
8662
8663
8664
8665
8666
8667
8668
8669
8671
8672
8673
8674
8675
8677
8678
8679
8680
8681
8682
8683
8684
8685
8686
8687
8689
8690
8691
8692
8693
8695
8697
8699
8700
8702
8704
8705
8707
8708
8709
8710
8713
8714
8717
8718
8719
8721
8722
8723
8725
8728
8729
8731
8732
8734
8735
8736
8737
8738
8739
8740
8741
8742
8743
8744
8745
8746
8748
8749
8750
8751
8752
8753
8754
8755
8756
8757
8760
8761
8762
8763
8764
8765
8766
8767
8768
8769
8770
8771
8772
8773
8774
8775
8776
8777
8778
8779
8780
8781
8782
8783
8784
8785
8786
8787
8788
8789
8790
8791
8794
8795
8796
8797
8799
8801
8802
8803
8804
8805
8806
8807
8808
8809
8810
8811
8812


11080
11081
11082
11084
11085
11086
11087
11089
11090
11091
11092
11093
11095
11096
11099
11101
11102
11103
11104
11105
11106
11108
11109
11110
11111
11112
11113
11114
11115
11117
11118
11119
11120
11121
11123
11126
11127
11128
11131
11132
11133
11134
11137
11138
11139
11140
11141
11142
11143
11146
11148
11149
11150
11151
11153
11154
11155
11156
11157
11158
11159
11160
11161
11162
11163
11164
11165
11167
11168
11170
11171
11172
11173
11174
11175
11177
11178
11180
11181
11183
11186
11189
11191
11195
11196
11198
11199
11200
11201
11202
11203
11204
11205
11207
11310
11311
11312
11313
11314
11315
11317
11318
11319
11320
11321
11322
11323
11324
11325
11326
11328
11329
11330
11332
11336
11337
11338
11339
11340
11341
11342
11343
11344
11345
11347
11348
11350
11351
11352
11353
11354
11355
11356
11357
11358
11359
11360
11361
11362
11363
11364
11365
11366
11367
11368
11371
11372
11373
11375
11376
11377
11380
11381
11382
11384
11387
11388
11389
11390
11391
11392
11393
11394
11396
11397
11398
1139

In [21]:
len(trouble_indices)

5938

In [23]:
high_ess = new_data[new_data['high_ess'] == 1]
high_ess

,CMO_NUMSTATES,CMO_SIZE,LEVEL,LOCALE15,PVI2017,PVI2014,RLAPROF_1516,MTHPROF_1516,ess_strength,prog_strength,...,RIT_STR,NUMWORDS,PROG_NORM,ESS_NORM,RIT_NORM,PCT_CHILDPOV_SAIPE16,high_ess,high_prog,high_both,low_both
22,1.0,3494.0,1,City (large),-15.0,-13.0,44.0,17.0,NaN,NaN,...,-1.831230,339,0.0,0.829002,0.852981,NaN,1,0,0,0
23,1.0,5313.0,1,City (large),-15.0,-13.0,44.0,17.0,NaN,NaN,...,-1.831230,339,0.0,0.829002,0.852981,NaN,1,0,0,0
28,1.0,3494.0,1,City (large),-15.0,-13.0,58.0,35.0,NaN,NaN,...,-1.846278,2948,0.0,0.837393,0.849902,NaN,1,0,0,0
243,3.0,5262.0,4,City (large),-34.0,-32.0,80.0,80.0,0.105652,0.097081,...,-1.782950,365,0.0,0.821387,0.862860,NaN,1,0,0,0
386,1.0,1668.0,1,City (large),-36.0,-35.0,NaN,NaN,0.136364,0.159091,...,-1.625312,422,0.0,0.807542,0.895114,NaN,1,0,0,0
595,1.0,904.0,1,City (large),-35.0,-30.0,54.0,45.0,0.139608,0.155815,...,-2.499740,8221,0.0,0.802612,0.716196,0.272314,1,0,0,0
1082,1.0,2481.0,1,Suburb (large),9.0,8.0,29.0,39.0,NaN,NaN,...,-6.000000,163,0.0,0.832169,0.000000,NaN,1,0,0,0
1083,1.0,NaN,1,Suburb (large),9.0,8.0,29.0,39.0,NaN,NaN,...,-6.000000,163,0.0,0.832169,0.000000,NaN,1,0,0,0
1084,1.0,2481.0,1,City (large),7.0,8.0,29.0,29.0,NaN,NaN,...,-6.000000,163,0.0,0.832169,0.000000,NaN,1,0,0,0
1085,1.0,NaN,1,City (large),7.0,8.0,29.0,29.0,NaN,NaN,...,-6.000000,163,0.0,0.832169,0.000000,NaN,1,0,0,0


In [24]:
high_prog = new_data[new_data['high_prog'] == 1]
high_prog

,CMO_NUMSTATES,CMO_SIZE,LEVEL,LOCALE15,PVI2017,PVI2014,RLAPROF_1516,MTHPROF_1516,ess_strength,prog_strength,...,RIT_STR,NUMWORDS,PROG_NORM,ESS_NORM,RIT_NORM,PCT_CHILDPOV_SAIPE16,high_ess,high_prog,high_both,low_both
306,1.0,758.0,1,City (small),21.0,20.0,39.0,39.0,0.050000,0.200000,...,-2.243038,351,0.776636,0.0,0.768720,NaN,0,1,0,0
469,1.0,NaN,1,City (large),-26.0,-24.0,69.0,14.0,NaN,NaN,...,-1.931966,1031,0.804452,0.0,0.832369,0.308620,0,1,0,0
614,1.0,658.0,1,City (large),-44.0,-43.0,NaN,NaN,0.033898,0.084746,...,-1.638783,741,0.771647,0.0,0.892358,NaN,0,1,0,0
615,1.0,NaN,1,City (large),-44.0,-43.0,NaN,NaN,0.033898,0.084746,...,-1.638783,741,0.771647,0.0,0.892358,NaN,0,1,0,0
661,1.0,1454.0,1,City (large),NaN,NaN,24.0,19.0,0.061249,0.179584,...,-2.033926,866,0.777980,0.0,0.811507,NaN,0,1,0,0
664,1.0,1454.0,1,City (large),NaN,NaN,14.0,69.0,0.039683,0.277778,...,-1.831928,749,0.770922,0.0,0.852838,NaN,0,1,0,0
703,1.0,7063.0,2,City (large),-7.0,-13.0,59.0,49.0,0.024194,0.088710,...,-1.955447,724,0.773858,0.0,0.827565,0.189168,0,1,0,0
852,1.0,NaN,4,Suburb (large),-40.0,-37.0,56.0,37.0,0.166667,0.833333,...,-1.655619,183,0.773115,0.0,0.888913,0.099929,0,1,0,0
1548,1.0,NaN,4,Rural (fringe),-14.0,-10.0,38.0,18.0,NaN,NaN,...,-2.324282,211,0.990115,0.0,0.752096,0.116505,0,1,0,0
1550,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.292256,196,0.996735,0.0,0.758649,0.324468,0,1,0,0


In [25]:
high_both = new_data[new_data['high_both'] == 1]
high_both

,CMO_NUMSTATES,CMO_SIZE,LEVEL,LOCALE15,PVI2017,PVI2014,RLAPROF_1516,MTHPROF_1516,ess_strength,prog_strength,...,RIT_STR,NUMWORDS,PROG_NORM,ESS_NORM,RIT_NORM,PCT_CHILDPOV_SAIPE16,high_ess,high_prog,high_both,low_both
509,1.0,542.0,4,Suburb (large),-14.0,-15.0,59.0,24.0,0.230769,0.179487,...,-1.718086,1467,0.772869,0.820931,0.876131,NaN,0,0,1,0
541,1.0,NaN,3,Suburb (large),15.0,17.0,39.0,29.0,0.172222,0.200000,...,-1.373372,380,0.805654,0.817545,0.946664,NaN,0,0,1,0
542,1.0,686.0,3,Suburb (large),15.0,17.0,39.0,29.0,0.172222,0.200000,...,-1.373372,380,0.805654,0.817545,0.946664,NaN,0,0,1,0
648,1.0,NaN,1,City (large),-38.0,-36.0,14.0,10.0,0.086093,0.370861,...,-2.539468,11086,0.794270,0.840228,0.708067,0.265633,0,0,1,0
652,1.0,960.0,1,City (large),-29.0,-25.0,44.0,59.0,0.088235,0.044118,...,-6.000000,140,0.797313,0.846682,0.000000,0.272314,0,0,1,0
1042,1.0,NaN,3,City (large),-17.0,-12.0,49.0,59.0,0.172414,0.034483,...,-1.766793,10791,0.787154,0.847922,0.866166,NaN,0,0,1,0
1043,1.0,NaN,3,City (large),-17.0,-12.0,54.0,89.0,0.172414,0.034483,...,-1.766793,10791,0.787154,0.847922,0.866166,NaN,0,0,1,0
1120,1.0,550.0,3,Suburb (large),-29.0,-26.0,74.0,34.0,NaN,NaN,...,-2.035287,53308,0.800067,0.828893,0.811228,0.245650,0,0,1,0
1132,1.0,512.0,1,City (large),-4.0,1.0,33.0,53.0,NaN,NaN,...,-1.850384,993,0.781828,0.830717,0.849062,NaN,0,0,1,0
1381,1.0,NaN,1,Suburb (large),-32.0,-30.0,24.0,19.0,0.076132,0.152263,...,-1.743454,1554,0.804016,0.815434,0.870941,NaN,0,0,1,0


In [26]:
low_both = new_data[new_data['low_both'] == 1]
low_both

,CMO_NUMSTATES,CMO_SIZE,LEVEL,LOCALE15,PVI2017,PVI2014,RLAPROF_1516,MTHPROF_1516,ess_strength,prog_strength,...,RIT_STR,NUMWORDS,PROG_NORM,ESS_NORM,RIT_NORM,PCT_CHILDPOV_SAIPE16,high_ess,high_prog,high_both,low_both
4,1.0,NaN,3,City (large),9.0,12.0,59.0,49.0,0.083333,0.020833,...,-2.664642,462,0.0,0.0,0.682455,NaN,0,0,0,1
5,1.0,1220.0,3,City (large),9.0,12.0,59.0,49.0,0.083333,0.020833,...,-2.664642,462,0.0,0.0,0.682455,NaN,0,0,0,1
19,1.0,NaN,2,City (large),9.0,11.0,89.0,85.0,0.217391,0.413043,...,-2.586587,772,0.0,0.0,0.698426,NaN,0,0,0,1
21,1.0,NaN,1,City (large),9.0,11.0,92.0,86.0,0.217391,0.413043,...,-2.416641,522,0.0,0.0,0.733199,NaN,0,0,0,1
29,6.0,23788.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.000000,1,0.0,0.0,0.000000,0.165307,0,0,0,1
30,1.0,14951.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-6.000000,1,0.0,0.0,0.000000,0.165307,0,0,0,1
31,6.0,23788.0,3,Town (remote),-6.0,1.0,59.0,49.0,NaN,NaN,...,-6.000000,1,0.0,0.0,0.000000,0.175336,0,0,0,1
32,1.0,14951.0,3,Town (remote),-6.0,1.0,59.0,49.0,NaN,NaN,...,-6.000000,1,0.0,0.0,0.000000,0.175336,0,0,0,1
33,6.0,23788.0,1,Suburb (large),-11.0,-9.0,94.0,95.0,NaN,NaN,...,-6.000000,1,0.0,0.0,0.000000,0.165307,0,0,0,1
34,1.0,14951.0,1,Suburb (large),-11.0,-9.0,94.0,95.0,NaN,NaN,...,-6.000000,1,0.0,0.0,0.000000,0.165307,0,0,0,1


In [27]:
final_sample = high_ess.sample(25)

In [28]:
final_sample

,CMO_NUMSTATES,CMO_SIZE,LEVEL,LOCALE15,PVI2017,PVI2014,RLAPROF_1516,MTHPROF_1516,ess_strength,prog_strength,...,RIT_STR,NUMWORDS,PROG_NORM,ESS_NORM,RIT_NORM,PCT_CHILDPOV_SAIPE16,high_ess,high_prog,high_both,low_both
7518,NaN,NaN,1,City (large),-20.0,-16.0,39.0,29.0,0.220833,0.158333,...,-1.950140,2319,0.0,0.849159,0.828650,0.220409,1,0,0,0
3374,NaN,NaN,N,City (large),-27.0,-24.0,NaN,NaN,NaN,NaN,...,-1.602060,640,0.0,0.931360,0.899872,NaN,1,0,0,0
3258,NaN,NaN,N,City (large),-38.0,-36.0,NaN,NaN,0.684211,0.039474,...,-1.450469,398,0.0,0.998794,0.930889,0.265633,1,0,0,0
1845,1.0,NaN,2,Rural (fringe),-7.0,-5.0,NaN,NaN,0.666667,0.038462,...,-1.448265,396,0.0,0.999274,0.931340,0.229292,1,0,0,0
2451,1.0,9208.0,1,City (midsize),25.0,24.0,89.0,84.0,0.250000,0.250000,...,-6.000000,90,0.0,0.888838,0.000000,NaN,1,0,0,0
7196,NaN,NaN,1,Suburb (large),-14.0,-10.0,83.0,68.0,0.157618,0.159370,...,-6.000000,208,0.0,0.808908,0.000000,0.154841,1,0,0,0
1084,1.0,2481.0,1,City (large),7.0,8.0,29.0,29.0,NaN,NaN,...,-6.000000,163,0.0,0.832169,0.000000,NaN,1,0,0,0
9095,NaN,NaN,4,City (midsize),6.0,4.0,50.0,50.0,0.158730,0.063492,...,-6.000000,488,0.0,0.832364,0.000000,NaN,1,0,0,0
3079,1.0,12821.0,4,City (large),-19.0,-12.0,63.0,61.0,NaN,NaN,...,-2.243440,6482,0.0,0.814372,0.768638,NaN,1,0,0,0
7434,NaN,NaN,3,City (large),-35.0,-30.0,79.0,39.0,0.246154,0.046154,...,-1.649984,269,0.0,0.850506,0.890066,0.272314,1,0,0,0


In [31]:
final_sample = final_sample.append(high_prog.sample(25))

In [32]:
final_sample

,CMO_NUMSTATES,CMO_SIZE,LEVEL,LOCALE15,PVI2017,PVI2014,RLAPROF_1516,MTHPROF_1516,ess_strength,prog_strength,...,RIT_STR,NUMWORDS,PROG_NORM,ESS_NORM,RIT_NORM,PCT_CHILDPOV_SAIPE16,high_ess,high_prog,high_both,low_both
7518,NaN,NaN,1,City (large),-20.0,-16.0,39.0,29.0,0.220833,0.158333,...,-1.950140,2319,0.000000,0.849159,0.828650,0.220409,1,0,0,0
3374,NaN,NaN,N,City (large),-27.0,-24.0,NaN,NaN,NaN,NaN,...,-1.602060,640,0.000000,0.931360,0.899872,NaN,1,0,0,0
3258,NaN,NaN,N,City (large),-38.0,-36.0,NaN,NaN,0.684211,0.039474,...,-1.450469,398,0.000000,0.998794,0.930889,0.265633,1,0,0,0
1845,1.0,NaN,2,Rural (fringe),-7.0,-5.0,NaN,NaN,0.666667,0.038462,...,-1.448265,396,0.000000,0.999274,0.931340,0.229292,1,0,0,0
2451,1.0,9208.0,1,City (midsize),25.0,24.0,89.0,84.0,0.250000,0.250000,...,-6.000000,90,0.000000,0.888838,0.000000,NaN,1,0,0,0
7196,NaN,NaN,1,Suburb (large),-14.0,-10.0,83.0,68.0,0.157618,0.159370,...,-6.000000,208,0.000000,0.808908,0.000000,0.154841,1,0,0,0
1084,1.0,2481.0,1,City (large),7.0,8.0,29.0,29.0,NaN,NaN,...,-6.000000,163,0.000000,0.832169,0.000000,NaN,1,0,0,0
9095,NaN,NaN,4,City (midsize),6.0,4.0,50.0,50.0,0.158730,0.063492,...,-6.000000,488,0.000000,0.832364,0.000000,NaN,1,0,0,0
3079,1.0,12821.0,4,City (large),-19.0,-12.0,63.0,61.0,NaN,NaN,...,-2.243440,6482,0.000000,0.814372,0.768638,NaN,1,0,0,0
7434,NaN,NaN,3,City (large),-35.0,-30.0,79.0,39.0,0.246154,0.046154,...,-1.649984,269,0.000000,0.850506,0.890066,0.272314,1,0,0,0


In [33]:
final_sample = final_sample.append(high_both.sample(25))

In [34]:
final_sample = final_sample.append(low_both.sample(25))

In [35]:
final_sample

,CMO_NUMSTATES,CMO_SIZE,LEVEL,LOCALE15,PVI2017,PVI2014,RLAPROF_1516,MTHPROF_1516,ess_strength,prog_strength,...,RIT_STR,NUMWORDS,PROG_NORM,ESS_NORM,RIT_NORM,PCT_CHILDPOV_SAIPE16,high_ess,high_prog,high_both,low_both
7518,NaN,NaN,1,City (large),-20.0,-16.0,39.0,29.0,0.220833,0.158333,...,-1.950140,2319,0.000000,0.849159,0.828650,0.220409,1,0,0,0
3374,NaN,NaN,N,City (large),-27.0,-24.0,NaN,NaN,NaN,NaN,...,-1.602060,640,0.000000,0.931360,0.899872,NaN,1,0,0,0
3258,NaN,NaN,N,City (large),-38.0,-36.0,NaN,NaN,0.684211,0.039474,...,-1.450469,398,0.000000,0.998794,0.930889,0.265633,1,0,0,0
1845,1.0,NaN,2,Rural (fringe),-7.0,-5.0,NaN,NaN,0.666667,0.038462,...,-1.448265,396,0.000000,0.999274,0.931340,0.229292,1,0,0,0
2451,1.0,9208.0,1,City (midsize),25.0,24.0,89.0,84.0,0.250000,0.250000,...,-6.000000,90,0.000000,0.888838,0.000000,NaN,1,0,0,0
7196,NaN,NaN,1,Suburb (large),-14.0,-10.0,83.0,68.0,0.157618,0.159370,...,-6.000000,208,0.000000,0.808908,0.000000,0.154841,1,0,0,0
1084,1.0,2481.0,1,City (large),7.0,8.0,29.0,29.0,NaN,NaN,...,-6.000000,163,0.000000,0.832169,0.000000,NaN,1,0,0,0
9095,NaN,NaN,4,City (midsize),6.0,4.0,50.0,50.0,0.158730,0.063492,...,-6.000000,488,0.000000,0.832364,0.000000,NaN,1,0,0,0
3079,1.0,12821.0,4,City (large),-19.0,-12.0,63.0,61.0,NaN,NaN,...,-2.243440,6482,0.000000,0.814372,0.768638,NaN,1,0,0,0
7434,NaN,NaN,3,City (large),-35.0,-30.0,79.0,39.0,0.246154,0.046154,...,-1.649984,269,0.000000,0.850506,0.890066,0.272314,1,0,0,0


In [37]:
final_sample[final_sample['high_ess'] == 1].shape[0]

25

In [38]:
final_sample[final_sample['high_prog'] == 1].shape[0]

25

In [39]:
final_sample[final_sample['high_both'] == 1].shape[0]

25

In [40]:
final_sample[final_sample['low_both'] == 1].shape[0]

25

In [41]:
final_sample.to_csv(folder_prefix + "random_sample_100.csv", mode="w", index=False, header=final_sample.columns.values, sep="\t", encoding="utf-8")

In [42]:
k = pd.read_csv(folder_prefix+ 'random_sample_100.csv' , sep="\t", low_memory=False, encoding="utf-8")
k

,CMO_NUMSTATES,CMO_SIZE,LEVEL,LOCALE15,PVI2017,PVI2014,RLAPROF_1516,MTHPROF_1516,ess_strength,prog_strength,...,RIT_STR,NUMWORDS,PROG_NORM,ESS_NORM,RIT_NORM,PCT_CHILDPOV_SAIPE16,high_ess,high_prog,high_both,low_both
0,NaN,NaN,1,City (large),-20.0,-16.0,39.0,29.0,0.220833,0.158333,...,-1.950140,2319,0.000000,0.849159,0.828650,0.220409,1,0,0,0
1,NaN,NaN,N,City (large),-27.0,-24.0,NaN,NaN,NaN,NaN,...,-1.602060,640,0.000000,0.931360,0.899872,NaN,1,0,0,0
2,NaN,NaN,N,City (large),-38.0,-36.0,NaN,NaN,0.684211,0.039474,...,-1.450469,398,0.000000,0.998794,0.930889,0.265633,1,0,0,0
3,1.0,NaN,2,Rural (fringe),-7.0,-5.0,NaN,NaN,0.666667,0.038462,...,-1.448265,396,0.000000,0.999274,0.931340,0.229292,1,0,0,0
4,1.0,9208.0,1,City (midsize),25.0,24.0,89.0,84.0,0.250000,0.250000,...,-6.000000,90,0.000000,0.888838,0.000000,NaN,1,0,0,0
5,NaN,NaN,1,Suburb (large),-14.0,-10.0,83.0,68.0,0.157618,0.159370,...,-6.000000,208,0.000000,0.808908,0.000000,0.154841,1,0,0,0
6,1.0,2481.0,1,City (large),7.0,8.0,29.0,29.0,NaN,NaN,...,-6.000000,163,0.000000,0.832169,0.000000,NaN,1,0,0,0
7,NaN,NaN,4,City (midsize),6.0,4.0,50.0,50.0,0.158730,0.063492,...,-6.000000,488,0.000000,0.832364,0.000000,NaN,1,0,0,0
8,1.0,12821.0,4,City (large),-19.0,-12.0,63.0,61.0,NaN,NaN,...,-2.243440,6482,0.000000,0.814372,0.768638,NaN,1,0,0,0
9,NaN,NaN,3,City (large),-35.0,-30.0,79.0,39.0,0.246154,0.046154,...,-1.649984,269,0.000000,0.850506,0.890066,0.272314,1,0,0,0
